In [3]:
import os
import cv2
import numpy as np
import ipywidgets as ipyw
import lz4.frame
import pandas as pd

import matplotlib

from matplotlib import pyplot as plt
%matplotlib inline

from werdich_cfr.utils.visuals import ImageSliceViewer3D

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [15]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
meta_date = '200202'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+meta_date)
cfr_meta_file = 'tfr_files_dset_BWH_'+meta_date+'.parquet'
meta_df = pd.read_parquet(os.path.join(meta_dir, cfr_meta_file))
max_samples_per_file = 15

# This should give us ~75% qualified files
min_rate = 20 # Minimum acceptable frame rage [fps]
min_frames = 30 # Minimum number of frames at min_rate (1.5 s)
max_frame_time = 1/min_rate*1e3 # Maximum frame time [ms]

view_list = sorted(list(meta_df.max_view.unique()))
mode_list = sorted(list(meta_df.dset.unique()))

# LOOP 1: VIEWS
view = view_list[2]

# LOOP 2: MODE
mode = mode_list[2]

# Filter view, mode and rates. Shuffle.
df = meta_df[(meta_df.max_view == view) & (meta_df.dset == mode) & (meta_df.frame_time < max_frame_time)].\
                                                                                            sample(frac=1)
print('View:{}, mode:{}, min_rate:{}, n_videos:{}'.format(view, mode, min_rate, len(df.filename.unique())))

# LOOP 3: FILES loop over all file names
filename = list(df.filename.unique())[100]
ser = df.loc[df.filename == filename, :].iloc[0]
file = os.path.join(ser.dir, filename)

try:
    with lz4.frame.open(file, 'rb') as fp:
        data = np.load(fp)

except IOError as err:
    print('Could not open this file: {}\n {}'.format(file, err))

print()
print(file)
print()
print(data.shape)

View:view_a4c, mode:train, min_rate:20, n_videos:10677

/mnt/obi0/phi/echo/npyFiles/BWH/4b72/4b729ab7c8fbe1dc_4903a581a72ff08f5060c835fa63/4b729ab7c8fbe1dc_4903a581a72ff08f5060c835fa63_Image-44.npy.lz4

(89, 600, 800, 1)


In [17]:
def im_scale(im):
    """ convert single images to uint8 and contrast en"""
    # We can do other things here: e.g. background subtraction or contrast enhancement
    im_scaled = np.uint8((im - np.amin(im))/(np.amax(im) - np.amin(im))*256)
    #im_scaled_eq = cv2.equalizeHist(im_scaled)
    return im_scaled

def data2imarray(im_data):
    """
    apply imscale function to np.array
    arg: im_array (frame, height, width)
    returns: im_array (height, width, frame)
    """
    im_data = np.squeeze(im_data)
    im_list = [im_scale(im_data[im]) for im in range(im_data.shape[0])]
    im_array = np.array(im_list, dtype=np.uint16)
    im_array = np.moveaxis(im_array, 0, -1)
    return im_array

im_array = data2imarray(data)
print(im_array.shape)

(600, 800, 89)


In [20]:
# subsampling to have the same frame rate

def subsample_time_index_list(frame_time, default_rate, min_frames):
    """
    rate: data frame rate,
    default_rate: desired frame rate,
    n_frames: number frames in the default rate (30)
    """
    default_times = np.arange(0, min_frames, 1) / default_rate
    times = np.arange(0, default_times[-1] + frame_time, frame_time)
    time_index_list = [np.argmin(np.abs(times - t)) for t in default_times]

    return time_index_list

def subsample_video(image_array, frame_time, min_rate, min_frames):
    """
    Select frames that are closest to a constant frame rate
    arg: image_array: np.array() [rows, columns, frame]
    """
    convert_video = True
    rate = 1 / frame_time
    # Check if the video is long enough
    min_video_len = min_frames / min_rate
    video_len = image_array.shape[-1] / rate
    if (min_video_len <= video_len) & (min_rate < rate):
        # print('Video is long enough and the rate is good.')
        # Get the frame index list
        time_index_list = subsample_time_index_list(frame_time=frame_time,
                                                    default_rate=min_rate,
                                                    min_frames=min_frames)
        # Select the frames from the video
        image_array = image_array[:, :, time_index_list]
    else:
        print('Frame rate: {:.1f} fps, length: {:.1f} s. Skipping.'.format(rate, video_len))
        convert_video = False

    return convert_video, image_array

In [38]:
frame_time

0.033333

In [21]:
min_rate = 20 # Minimum acceptable frame rage [fps]
min_frames = 30 # Minimum number of frames at min_rate (1.5 s)
convert_video, image_array = subsample_video(im_array, 
                                             frame_time = frame_time, 
                                             min_rate = min_rate, 
                                             min_frames = min_frames)
print(convert_video)
print(image_array.shape)

True
(600, 800, 30)


In [37]:
ImageSliceViewer3D(image_array)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…